In [9]:
# Install a conda package in the current Jupyter kernel
import sys

In [ ]:
!conda install --yes --prefix {sys.prefix} numpy

In [256]:
import numpy as np

class softmax_layer:
    def dim_in(self):
        return 0
    
    def dim_out(self):
        return 0
    
    def update_weigths(self, gradient, learning_rate):
        return gradient
    
    def __call__(self, x):
        exp = np.exp(x.T - np.max(x, axis=1))
        return (exp / np.sum(exp, axis=0))

In [257]:
class linear_layer:
    def __init__(self, dim_in, dim_out):
        self._A = np.random.rand(dim_in,dim_out)
        self._B = np.random.rand(dim_out)
        self._dim_in = dim_in
        self._dim_out = dim_out
        
    def update_weigths(self, gradient, learning_rate):
        self._A -= learning_rate * gradient
        return gradient
    
    def dim_in(self):
        return self._dim_in
    
    def dim_out(self):
        return self._dim_out
    
    def __call__(self, x):
        return x.T.dot(self._A) + self._B

In [258]:
def cross_entropy_loss(y_true, y_predicted, epsilon=1e-10):
    predictions = np.clip(y_predicted, epsilon, 1. - epsilon)
    N = predictions.shape[0]
    return -np.sum(y_true * np.log(predictions)) / N

In [259]:
class sequential_model:
    def __init__(self, *layers, learning_rate=0.01):
        self._learning_rate = learning_rate
        self._layers = []
        last_dim_out = 0
        for layer in layers:
            if last_dim_out != 0 and layer.dim_in() != 0 and last_dim_out != layer.dim_in():
                print('dimension dont match layer out dim {} , next layer dim in {}'.format(last_dim_out, layer.dim_in()))
                raise 
            self._layers.append(layer)
            if layer.dim_out() != 0:
                last_dim_out = layer.dim_out()
                
    def update_weigths(self, gradient):
        for layer in reversed(self._layers):
            gradient = layer.update_weigths(gradient, self._learning_rate)
            
    def __call__(self, x):
        for layer in self._layers:
            x = layer(x)
        return x

In [266]:
model = sequential_model(
        linear_layer(2, 2),
        softmax_layer()
        )

In [230]:
!conda install --yes --prefix {sys.prefix} scipy

Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.9.1
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda




In [231]:
import scipy.io
mat = scipy.io.loadmat('SwissRollData.mat')

In [261]:
X = mat['Yt']
Y = mat['Ct']
print(X.shape)

(2, 20000)


In [267]:
Y_predicted = model(X)
print(Y.shape)
print(Y_predicted.shape)

(2, 20000)
(2, 20000)


In [268]:
print(cross_entropy_loss(Y, Y_predicted))

7650.525471345105


In [269]:
def softmax_gradient(X, Y_true, Y_predicted):
    M = X.shape[0]
    return 1/M * X.dot((Y_true - Y_predicted).T)

In [270]:
print(softmax_gradient(X, Y, Y_predicted))

def SGD(model, X, Y):
    Y_predicted = model(X)
    gradient = softmax_gradient(X, Y, Y_predicted)
    model.update_weigths(gradient)

[[ 116.2355859  -116.2355859 ]
 [  74.67569786  -74.67569786]]


In [271]:
SGD(model, X, Y)

bla
bla
[[0.29429512 0.51109954]
 [0.67999543 0.54747531]]
[[-0.86806074  1.6734554 ]
 [-0.06676155  1.29423229]]


In [272]:
Y_predicted = model(X)
print(Y_predicted.T)
print(cross_entropy_loss(Y, Y_predicted))

[[0.69644951 0.30355049]
 [0.68110041 0.31889959]
 [0.69554053 0.30445947]
 ...
 [0.09410854 0.90589146]
 [0.09259983 0.90740017]
 [0.1115786  0.8884214 ]]
9503.22830217776
